In [1]:
import os
import sys
import math
import random
import pickle
import warnings

from datetime import datetime, timedelta
from bisect import bisect_left
from itertools import combinations
from math import sqrt

import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.tseries.offsets import MonthEnd
from matplotlib.patches import Rectangle

from scipy import stats
from scipy.stats import (norm, skew, kurtosis, pearsonr, spearmanr, kendalltau)
from scipy.optimize import curve_fit

from sklearn.metrics import mean_squared_error

import pysal
from numpy.random import Generator, PCG64
from functools import partial

import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from PIL import Image
from datetime import datetime, timedelta
#!pip install docx
from docx import Document

import pandas as pd

from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
warnings.filterwarnings("ignore")


# Fix 'from docx import Document' import by removing the wrong package and installing the right one

import sys, os, pathlib, importlib.util, subprocess

def run(cmd):
    print(f"\n=== $ {cmd}")
    return subprocess.call(cmd, shell=True)

print("Python executable:", sys.executable)
print("Active environment site-packages paths (top 5):", sys.path[:5])

# 1) Uninstall the wrong package ('docx') if present
run("pip uninstall -y docx")

# 2) Install the correct package ('python-docx')
# Prefer conda if you're using Anaconda; fall back to pip if conda isn't available.
if shutil := importlib.util.find_spec("shutil"):
    import shutil as _sh
    if _sh.which("conda"):
        run("conda install -y -c conda-forge python-docx")
    else:
        run("pip install -U python-docx")
else:
    # very defensive fallback
    run("pip install -U python-docx")

# 3) Guard against a local file shadowing the package (e.g., a file named 'docx.py' in your folder)
cwd_files = {p.name for p in pathlib.Path('.').iterdir() if p.is_file()}
if "docx.py" in cwd_files or "docx" in cwd_files:
    print("\n⚠️ Detected a local file named 'docx.py' or 'docx' in your working directory.")
    print("   This will shadow the 'python-docx' package. Rename that file and re-run this cell.")

# 4) Test the import
print("\n=== Import test ===")
try:
    from docx import Document
    print("✅ Import succeeded. Version check below:")
    import docx
    print("python-docx version:", getattr(docx, "__version__", "unknown"))
except Exception as e:
    print("❌ Import still failing:", repr(e))
    print("If this persists, restart the kernel (Kernel → Restart) and run this cell again.")


# CREATE A FUNCTION THAT SLICES THE DATA

In [20]:
def filter_dataframe_by_months(dataframe, months):
    """
    Filters rows from a DataFrame based on the month of its DateTime index.

    Parameters:
    - dataframe (pd.DataFrame): The DataFrame to filter. Must have a DateTime index.
    - months (list of int): List of months to filter for (e.g., [1, 2, 3] for Jan, Feb, Mar).

    Returns:
    - pd.DataFrame: A DataFrame containing only the rows for the specified months.
    """
    # Validate input
    if not isinstance(months, list) or not all(isinstance(month, int) for month in months):
        raise ValueError("Months must be provided as a list of integers.")
    if not hasattr(dataframe.index, "month"):
        raise ValueError("The DataFrame must have a DateTime index.")

    # Perform filtering
    return dataframe[dataframe.index.month.isin(months)]


# rolling cross correlate

In [21]:

def rolling_cross_correlate(df1, df2, col1, col2, lag=-3, window=366):
    """
    Computes rolling cross-correlation between two time-series columns from 
    two datetime-indexed DataFrames with a specific lag.

    Parameters:
    -----------
    df1 : pd.DataFrame
        First DataFrame with a datetime index.
    df2 : pd.DataFrame
        Second DataFrame with a datetime index.
    col1 : str
        Column name in the first DataFrame to use for correlation.
    col2 : str
        Column name in the second DataFrame to use for correlation.
    lag : int, optional
        Lag (in time steps) to consider. Default is -3.
    window : int, optional
        Rolling window size (in days) for the correlation. Default is 366.

    Returns:
    --------
    rolling_correlations : pd.Series
        Rolling cross-correlation values for the specified lag.
    """

    # Drop rows with NaN values
    df1 = df1.dropna()
    df2 = df2.dropna()
    
    # Align data based on datetime index
    merged = pd.merge(df1[[col1]], df2[[col2]], left_index=True, right_index=True, how='inner')

    # Shift the second series by the specified lag
    shifted_series2 = merged[col2].shift(-lag)
    
    # Calculate rolling correlation
    rolling_correlations, p_value = merged[[col1, col2]].rolling(window=window).corr(shifted_series2).iloc[:, 0]

    return rolling_correlations

# 2. ADJUST ZAK BAILLIES SIGNIFICANT TEST P-VALE FUNCTION

In [22]:
#Defines the correlation value based on a Pearsons principle and also checks whether the significance of the relationship is true on a 
#95% and 99% level
def corr_sig(x, y):
    """
    Calculate Pearson correlation and test if p-value <= 0.05.
    
    Returns:
        corr (float): Correlation coefficient
        is_significant (bool): True if p <= 0.01, else False, whcih is the 99th percentile
    """
    corr, p_value = pearsonr(x, y)
    if p_value < 0.01:
        significance = '99%'  # 99% confidence
    elif p_value < 0.05:
        significance = '95%'   # 95% confidence
    else:
        significance = ''
    
    return pd.Series({
        'Correlation': corr.round(2),
        'p_value': p_value.round(2),
        'Significance': significance})


In [2]:

3 // 2 # gives the window to the lowest value - half a window is 1+5x2


1

# 10. Function For Rolling Correlation On a 366 Day Basis

In [23]:
def running_correlation_with_p(df, col1, col2, window, min_valid_ratio=0.7):
    """
    Calculate running centered correlation and significance between two columns.
    NaN if any value missing in the window or if too much data is missing.

    Args:
        df: DataFrame with DateTime index
        col1: First column name
        col2: Second column name
        window: Window size (must be odd)
        min_valid_ratio: Minimum proportion of valid data required to calculate correlation
    
    Returns:
        pd.DataFrame with 'correlation' and 'significant' columns
    """
    if window % 2 == 0:
        raise ValueError("Window size must be odd for centered correlation.")
    
    half_window = window // 2 # gives the window to the lowest value - half a window is 1+5x2
    corr_values = []
    sig_values = []

    for i in range(len(df)):
        if i - half_window < 0 or i + half_window >= len(df):
            corr_values.append(np.nan)
            sig_values.append(np.nan)
            continue
        sub = df.iloc[i - half_window : i + half_window + 1][[col1, col2]]
        valid = sub.dropna()
        if len(valid) / window < min_valid_ratio:  #This checks to see if avalaible data that can be used, removes data with less then 70%
            # Too much missing data
            corr_values.append(np.nan)
            sig_values.append(np.nan)
            continue

        corr, p_value = pearsonr(valid[col1], valid[col2])
        corr_values.append(corr)
        significance = (
            '95%' if p_value < 0.05 else
            '90%' if p_value < 0.1 else
            '')
        
        sig_values.append(significance)

    result = pd.DataFrame({
        'correlation': corr_values,
        'significant': sig_values
    }, index=df.index)

    return result


# WMO GUIDELINES MONTHLY FUNCTION

In [3]:
# WMO Guidelined Months
def WMO_Guide_Months(D,Date_Column,Type_Column,Raindays):    

    #The appendenation of the daily data into monthlies
    Usuable = [] #WMO Standard 
    #Since we have dropped all periods with missing days lets work out days avalaible
    count_for_month = D.resample('MS').count().reset_index()

    #Fill in all days where no data is avalaible with nans
    nan_dates = pd.date_range(start=D.index.min(), end=D.index.max(), freq='D') 
    # Create an empty DataFrame with these monthly dates as index and label it MSLP
    Data_Missing = pd.DataFrame(index=nan_dates, columns=[Type_Column])
    Data_Missing.index.name = Date_Column  # Set the index name to 'Date'
    Data_Missing[:] = np.nan
    D = pd.concat([D, Data_Missing[~Data_Missing.index.isin(D.index)]])
    D = D.sort_index()

    #Get it into monthlies
    if (Raindays == True):
        M = D.resample('MS').sum()
    else:
        M = D.resample('MS').mean()

    #Reset Index - easier to apply on the for loop
    M = M.reset_index()

    #Initial check is to remove monthlies where 11 days or more are missing
    for i in range(0, len(count_for_month)):
        #Get the year and month 
        Year = count_for_month[Date_Column].loc[i].year
        Month = count_for_month[Date_Column].loc[i].month

        #Check for leap year and total days
        Total_Monthly_Count = [31,29 if Year % 4 == 0 else 28,31,30,31,30,31,31,30,31,30,31]

        #Chekc the number of days avalaible for that month
        Count_Month_Total = Total_Monthly_Count[Month-1]
        
        #Check the count
        Count = count_for_month.loc[i].loc[Type_Column] #Type column is Rainfall/Raindays/MLSP
    
        #If the count is less then 11, then we keep it
        if Count < Count_Month_Total - 11: #11 randomly missing days
            x = 0
        else:
            #Now we check whether it has 5 or more consecutive days missing
            #Checks current count
            Consec_NaNs = 0
            #Checks max count of the month
            Consec_Max = 0
                
            Months_Data = D.loc["{}-{}".format(Year, Month)].reset_index() #Extract the months daily data
            
            for q in range(0, len(Months_Data)):
                #If daily data is a nan, it will add to Consec NaNs
                if np.isnan(Months_Data[Type_Column].loc[q]) == True:
                    Consec_NaNs = Consec_NaNs + 1
                else:
                    #if not a nan, it will save the nan check count
                    Consec_Check = Consec_NaNs
                    #The save check count is then checked to see if its > then the max of that month
                    if Consec_Check >= Consec_Max:
                        #If so then the max check count is then updated
                        Consec_Max = Consec_Check
                    Consec_NaNs = 0

            #Once the month has been checked - if the max check count is 5 or more, then its a nan
            if Consec_Max >= 5: # this chekcs to on more then 5 consecutive days
               x = 0
            else: 
                Usuable.append(M.loc[i])
    
    #####################
    # Convert list of dictionaries to DataFrame
    Usuable = pd.DataFrame(Usuable)
    
    # Convert 'Date' column to datetime
    Usuable[Date_Column] = pd.to_datetime(Usuable[Date_Column])
    
    # Set 'Date' column as index
    Usuable.set_index(Date_Column, inplace=True)

    #Ensure all monthly dates are filled with NaNs
    start_date = Usuable.index.min()
    end_date = Usuable.index.max()
    # Create a monthly date range
    monthly_dates = pd.date_range(start=start_date, end=end_date, freq='MS')  # 'MS' = Month Start
    # Create an empty DataFrame with these monthly dates as index and label it MSLP
    MSLP_Missing = pd.DataFrame(index=monthly_dates, columns=[Type_Column])
    MSLP_Missing.index.name = Date_Column  # Set the index name to 'Date'
    MSLP_Missing[:] = np.nan
    
    Usuable_infil = pd.concat([Usuable, MSLP_Missing[~MSLP_Missing.index.isin(Usuable.index)]])
    Usuable_infil = Usuable_infil.sort_index()
    
    return(Usuable_infil.round(2))


In [4]:
# WMO Guidelined Months
def WMO_Guide_Yearly(M,Date_Column,Type_Column,Raindays):
    Usuable = [] #WMO Standard 
    #The appendenation of the daily data into monthlies
    Usuable = [] #WMO Standard 
    #Since we have dropped all periods with missing days lets work out days avalaible
    count_for_year = M.resample('YS').count().reset_index()

    #Fill in all days where no data is avalaible with nans
    nan_dates = pd.date_range(start=M.index.min(), end=M.index.max(), freq='MS') 
    # Create an empty DataFrame with these monthly dates as index and label it MSLP
    Data_Missing = pd.DataFrame(index=nan_dates, columns=[Type_Column])
    Data_Missing.index.name = Date_Column  # Set the index name to 'Date'
    Data_Missing[:] = np.nan
    M = pd.concat([M, Data_Missing[~Data_Missing.index.isin(M.index)]])
    M = M.sort_index()

    #Get it into monthlies
    if (Raindays == True):
        Y = M.resample('YS').sum()
    else:
        Y = M.resample('YS').mean()

    #Reset Index - easier to apply on the for loop
    Y = Y.reset_index()

    #Initial check is to remove monthlies where 11 days or more are missing
    for i in range(0, len(count_for_year)):
        #Get the year and month 
        Year = count_for_year[Date_Column].loc[i].year

        #Check for leap year and total days
        Total_Yearly_Count = 12
        
        #Chekc the number of days avalaible for that month
        Count_Yearly_Total = Total_Yearly_Count
        
        #Check the count
        Count = count_for_year.loc[i].loc[Type_Column] #Type column is Rainfall/Raindays/MLSP
    
        #If the count is less then 11, then we keep it
        if Count < Count_Yearly_Total - 5: #5 randomly missing months
            x = 0
        else:
            #Now we check whether it has 5 or more consecutive days missing
            #Checks current count
            Consec_NaNs = 0
            #Checks max count of the month
            Consec_Max = 0
                
            Yearly_Data = M.loc["{}".format(Year)].reset_index() #Extract the months daily data
            
            for q in range(0, len(Yearly_Data)):
                #If daily data is a nan, it will add to Consec NaNs
                if np.isnan(Yearly_Data[Type_Column].loc[q]) == True:
                    Consec_NaNs = Consec_NaNs + 1
                else:
                    #if not a nan, it will save the nan check count
                    Consec_Check = Consec_NaNs
                    #The save check count is then checked to see if its > then the max of that month
                    if Consec_Check >= Consec_Max:
                        #If so then the max check count is then updated
                        Consec_Max = Consec_Check
                    Consec_NaNs = 0

            #Once the month has been checked - if the max check count is 5 or more, then its a nan
            if Consec_Max >= 3: # this chekcs to on more then 5 consecutive months
               x = 0
            else: 
                Usuable.append(Y.loc[i])
    
    #####################
    # Convert list of dictionaries to DataFrame
    Usuable = pd.DataFrame(Usuable)
    
    # Convert 'Date' column to datetime
    Usuable[Date_Column] = pd.to_datetime(Usuable[Date_Column])
    
    # Set 'Date' column as index
    Usuable.set_index(Date_Column, inplace=True)

    #Ensure all monthly dates are filled with NaNs
    start_date = Usuable.index.min()
    end_date = Usuable.index.max()
    # Create a monthly date range
    yearly_dates = pd.date_range(start=start_date, end=end_date, freq='YS')  # 'MS' = Month Start
    # Create an empty DataFrame with these monthly dates as index and label it MSLP
    MSLP_Missing = pd.DataFrame(index=yearly_dates, columns=[Type_Column])
    MSLP_Missing.index.name = Date_Column  # Set the index name to 'Date'
    MSLP_Missing[:] = np.nan
    
    Usuable_infil = pd.concat([Usuable, MSLP_Missing[~MSLP_Missing.index.isin(Usuable.index)]])
    Usuable_infil = Usuable_infil.sort_index()
    
    return(Usuable_infil.round(2))

# SUBDAILY TO YEARLY INCORPORATING WMO GUIDELINES

In [5]:
def SD_Y_WMO_GL(Data,Date_Col,Type_Col,Rain):
    if(Rain==True):
        #Raindays/Rainfall means no sub-dailies
        Data_D = Data.dropna() #Exclude all NaN data
        #Monthly data needs to be subjected to WMO guidelines
        Data_M = WMO_Guide_Months(Data_D,Date_Col,Type_Col,True)
        #Now apply similar guideline ratios onto the Yearly
        Data_Y = WMO_Guide_Yearly(Data_M.dropna(),Date_Col,Type_Col,True)
        return(Data_D,Data_M,Data_Y)
    else:
        #Pressure cna be averaged
        Data_SD = Data.dropna() #Exclude all NaN data
        Data_D = Data_SD.resample('D').mean()
        #Monthly data needs to be subjected to WMO guidelines
        Data_M = WMO_Guide_Months(Data_D,Date_Col,Type_Col,False)
        #Now apply similar guideline ratios onto the Yearly
        Data_Y = WMO_Guide_Yearly(Data_M.dropna(),Date_Col,Type_Col,False)
        return(Data_SD,Data_D,Data_M,Data_Y)




# UQC FUNCTIONS

In [26]:
#Zaks UQC format scripts turned into a single fucntion
def ymd_column(df):
    """adds a year month and day column to a given dataframe
    
    args: df containing datetime index
    """
    
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    df = df[cols]
    
    df = df.reset_index(drop=True)
    
    return df 


def UQC_pres_formats(df):
    data = df
    app_data = []
    for value in sorted(data.index.hour.unique().values):
        subset = data[data.index.hour == value]
        subset.index = subset.index.normalize()
        subset.columns = ['SLP{}'.format(str(value))]
        #Added in cause there was a duplicate somewhere
        subset = subset[~subset.index.duplicated(keep='first')]
        app_data.append(subset)
    
    UQC_formats = pd.concat(app_data,axis=1)   

    UQC_formats = ymd_column(UQC_formats)
    UQC_formats = UQC_formats.round(4)
    UQC_formats.fillna(-99.9,inplace=True)
    return UQC_formats

# FARHEINHEIGHT TO CELSUIS AND VICE VERSA

In [1]:
def f_to_c(temp):
    """Convert Fahrenheit to Celsius, ignore -99.9 (missing value)."""
    return round((temp - 32) * 5/9, 1)

def c_to_f(temp):
    """Convert Fahrenheit to Celsius, ignore -99.9 (missing value)."""
    return round((temp*(9/5)) + 32, 1)

# AS READ TO MSLP FUNCTIONS : 
## CORRECT TEMPERATURE
## GRAVITY CALCULATOR
## CORRECT GRAVITY


In [2]:
#Overall function version 2
def Reduce_to_MSLP_V2_2(Pressure_Data, Temperature_Data, 
                      Barometer_Type, Pressure_Unit, 
                      Temperature_Unit, latitude, 
                      elevation_feet, elevation_type):
    '''
    INPUTS:
    -------------------
    
    Pressure_Data : DATAFRAME
        Ensure the date is in the index column, pressure can be any of the typical units used, and make sure it
        has a label
        
    Temperature_Data : DATAFRAME
        Ensure the date is in the index column, temperature can be any of the typical units used, and make sure it
        has a label
    
    Barometer_Type : STRING
        What is the type of barometer used for this case. If you do not know use "Unknown", but the other barometer 
        types are "Fortin", "Cistern", "Fuess" and "Kew"
        
    Pressure_Unit : STRING
        What is the pressure units you use. It is "inHg", "mmHg", "mb", "bar", "hPa", 'Pa', or 'atm'
    
    Temperature_Unit : STRING
        What is the temperature units you use. It is "degF", "degC", or "K"
        
    latitude : RATIONAL NUMBER
        Where is the location of the barometer to best possible coordinate value you can find.
    
    elevation_feet : RATIONAL NUMBER (ft)
        What is the elevation of the barometer to the best possible knowledge in feet (ft).
        
    elevation_type : STRING
        What is the units of the elevation, is it meters/metres, m, ft or feet
        
    OUTPUTS:
    ---------------------
    
    Final_Product : DATAFRAME
        So this has all the necassary values such as MSLP in hPa which is the corrected to gravity and temperature,
        the temperature in degC, and the original pressure and adjusted to temperature pressure in the columns
        in that respective order.
    '''
    #Convert up the data so Pressure is in mb and temperature is in degC
    PT_Fixed = Converters(Pressure_Data,Temperature_Data,Pressure_Unit,Temperature_Unit)  
    if PT_Fixed == 'Error Not a Viable Pressure Unit':
        return('Error Not a Viable Pressure Unit')
    elif PT_Fixed == 'Error Not a Viable Temperature Unit':
        return('Error Not a Viable Temperature Unit')
    else:
        X = 0
    #Caculate the Barometer reading freduced to standard temperature but not to standard gravity
    Pres_T = Correct_Temp(PT_Fixed[0],PT_Fixed[1],Barometer_Type)  

    #Calculate GOH, this is local gravity for the barometer
    GOH,Elevation = calculate_gravity(latitude, elevation_feet,elevation_type)
    GOH = round(GOH,3)
    #Reduced to MSLP by fixing up pressure now by the gravity constants
    Pres_P = Correct_Pres(Pres_T,GOH) 
    
    #Now correct to MSLP
    Pres_P['MSLP'] = TO_MSLP(Pres_P, PT_Fixed[1],Elevation)
    Pres_P['MSLP'] = pd.to_numeric(Pres_P['MSLP'], errors='coerce')
    
    #Combine all the data together so you have Date|MSLP|degC|Raw Pressure|Reduced Temp Pressure
    Final_Product = pd.concat([Pres_P, PT_Fixed[1]],axis= 1)
    Final_Product['degC'] = pd.to_numeric(Final_Product['degC'], errors='coerce')
    Final_Product = round(Final_Product,2)
    Final_Product = Final_Product[['MSLP','degC',Final_Product.columns[0],Final_Product.columns[1],Final_Product.columns[2]]]
    return(Final_Product)


## REDUCE TEMPERATURE

def Correct_Temp(Pres,Temp,Barometer_Type): #WMO GUIDELINE 1971 3.2.6.3
    '''
    INPUTS:
    -----------------------------
    
    Pressure_Data : DATAFRAME
        Ensure the date is in the index column, pressure it is now in mb and ready to be corrected
        
    Temperature_Data : DATAFRAME
        Ensure the date is in the index column, temperature it is now in degC and ready to be corrected
        
    Barometer_Type : STRING
        What is the type of barometer used for this case. If you do not know use "Unknown", but the other barometer 
        types are "Fortin", "Cistern", "Fuess" and "Kew"
    
    OUTPUTS:
    ------------------------------
    Pres : DATAFRAME
        Dataframe with reduced to temperature pressure in millibars and also the original pressure is still kept as 
        well
    
    '''
    #If and else statements to ensure we have the barometer of choice, see WMO Guidelines 1971 for 
    #how these equations are built
    
    if Barometer_Type == 'Unknown':
        #Make it a fortin style barometer
        CT = -0.000163*Pres[Pres.columns[0]]*Temp[Temp.columns[0]]
        
    elif Barometer_Type == 'Fortin':
        CT = -0.000163*Pres[Pres.columns[0]]*Temp[Temp.columns[0]]
        
    elif Barometer_Type == 'Cistern':
        #Use the average value between Fuess and Kew Pattern if we do not know which one to take 
        CT = -0.000163*(Pres[Pres.columns[0]]+39)*Temp[Temp.columns[0]]
        
    elif Barometer_Type == 'Fuess':
        CT = -0.000163*(Pres[Pres.columns[0]]+31)*Temp[Temp.columns[0]]
        
    elif Barometer_Type == 'Kew':
        CT = -0.000163*(Pres[Pres.columns[0]]+47)*Temp[Temp.columns[0]]
        
    else:
        #In case there is an issue
        X = 'Error Not a Viable Barometer Type'
        return(X)
    
    #Update the column to include reduced temperature
    Pres['Reduced Temp Pressure (mb)'] = Pres[Pres.columns[0]] + CT
    
    return(Pres)


#Gravity calculator

def calculate_gravity(latitude, elevation, elevation_type): #WMO GUIDELINE 3.8
    
    '''
    INPUTS:
    --------------------
    
    latitude : RATIONAL NUMBER
        Where is the location of the barometer to best possible coordinate value you can find.
    
    elevation_feet : RATIONAL NUMBER
        What is the elevation of the barometer to the best possible knowledge in feet (ft)
    
    elevation_type : STRING
        What is the units of the elevation, is it meters/metres, m, ft or feet
        
    OUTPUTS:
    -------------------------------
    
    g : RATIONAL NUMBER
        Calculates the local gravity from the specific height in metres.
    
    '''

    # Convert feet to meters
    if elevation_type == 'feet':   
        elevation_meters = elevation * 0.3048
    elif elevation_type == 'metres':   
        elevation_meters = elevation
    elif elevation_type == 'meters':   
        elevation_meters = elevation
    elif elevation_type == 'ft':   
        elevation_meters = elevation * 0.3048
    elif elevation_type == 'm':   
        elevation_meters = elevation
    else:
        X = 'Not a viable unit for elevation'
        Y = 0
        return(X,Y)
    '''
    #Equation itself from original, now I have fixed up this gravity equation in the same form
    as the WMO guidelines, and assume H = H'
    A = 0.0026373#0.0053024
    B = 0.0000059#0.0000058

    L = math.radians(latitude)  # Convert latitude to radians
    #3.086e-6 
    g = (9.80616 * (1 + A * math.sin(L)**2 - B * math.sin(2 * L))) - (0.0003086 * elevation_meters)
    '''
    A = 0.0026373
    B = 0.0000059
    L = math.radians(latitude)
    
    g = (980.616 * (1 - (A * math.cos(2 * L)) + (B * math.cos(L)**2))) - (0.0003086 * elevation_meters)   
    
    return(g,elevation_meters)

#Reduce gravity
def Correct_Pres(Pres,GOH): #WMO GUIDELINE 1971 3.2.6.2
    '''
    INPUTS:
    --------------------
    
    Pres : DATAFRAME
        Dataframe with reduced to temperature pressure in millibars and also the original pressure is still kept as 
        well
    
    GOH : RATIONAL NUMBER
        Calculates the local gravity from the specific height in metres.    
    
    OUTPUTS:
    -------------------------------
    Pres : DATAFRAME
        Dataframe reduced to temperature and gravity in millibars, sicne its 1:1 with hPa, this means this is the 
        MSLP value to. Reduced to temperature pressure in millibars and also the original pressure is still kept as 
        well
    '''
    
    GN = 980.665 #cms-2 OUTLINED BY 3.2.6.2
    Pres['Correct_Pres_Temp'] = Pres[Pres.columns[1]]*(GOH/GN)#
    return(Pres)

#Fix up the gravity correction from the WMO guidelines - if only gravity correciton is required
def Correct_Pres_1_col_Only(Pres,GOH): #WMO GUIDELINE 1971 3.2.6.2
    GN = 980.665 #cms-2 OUTLINED BY 3.2.6.2
    Pres['Correct_temp_gravity_mslp'] = Pres[Pres.columns[0]]*(GOH/GN)#
    return(Pres)


# UNIT CONVERTORS FOR TEMPERATURE AND PRESSURE

In [29]:

###############################################
#Convert units to the WMO guidelines

def Converters(Pressure_Data,Temperature_Data,Pressure_Unit,Temperature_Unit):
    '''
    INPUTS:
    -------------------------
    
     Pressure_Data : DATAFRAME
        Ensure the date is in the index column, pressure can be any of the typical units used, and make sure it
        has a label
        
    Temperature_Data : DATAFRAME
        Ensure the date is in the index column, temperature can be any of the typical units used, and make sure it
        has a label
        
    Pressure_Unit : STRING
        What is the pressure units you use. It is "inHg", "mmHg", "mb", "bar", "hPa", 'Pa', or 'atm'
    
    Temperature_Unit : STRING
        What is the temperature units you use. It is "degF", "degC", or "K"
    
    OUTPUTS:
    ----------------------------
    
    Pressure_Data : DATAFRAME
        This is now an updated version of the pressure unit to WMO Guidelines for the implementation into the
        reduction to MSLP. This unit is mb
    
    Temperature_Data : DATAFRAME
        This is now an updated version of the temperature unit to WMO Guidelines for the implementation into the
        reduction to MSLP. This unit is degC
    '''
    #We need a bunch of if else statements
    #Pressure Conversion using if and else statement
    if Pressure_Unit == 'inHg':
        Pressure_Data = Pressure_Data*33.86 #In previous versions, this was 33.38338..., we have not restricted most values to 4-5 singificant figures
        
    elif Pressure_Unit == 'mmHg':
        Pressure_Data = Pressure_Data*1.333
        
    elif Pressure_Unit == 'bar':
        Pressure_Data = Pressure_Data*1000
    
    elif Pressure_Unit == 'mb':
        Pressure_Data = Pressure_Data*1
        
    elif Pressure_Unit == 'hPa':
        Pressure_Data = Pressure_Data*1

    elif Pressure_Unit == 'Pa':
        Pressure_Data = Pressure_Data*0.01

    elif Pressure_Unit == 'atm':
        Pressure_Data = Pressure_Data*1013.25 #only expection to the significant features rule
        
    else:
        #This cuts the function from progressing 
        X ='Error Not a Viable Pressure Unit'
        return(X)
    
    #Temperature Conversion
    if Temperature_Unit == 'degC':
        Temperature_Data = Temperature_Data*1
        
    elif Temperature_Unit == 'degF':
        Temperature_Data = (Temperature_Data-32)*(5/9)
        
    elif Temperature_Unit == 'K':
        Temperature_Data = Temperature_Data-273.15
        
    else:
        #This cuts the function from progressing 
        X = 'Error Not a Viable Temperature Unit'
        
        return(X)
   
    Pressure_Data.rename(columns = {Pressure_Data.columns[0]:'Raw mb'}, inplace= True)
    Temperature_Data.rename(columns = {Temperature_Data.columns[0]:'degC'}, inplace= True)
    
    #Put together to make it easier when displaying incorrect issues
    PT = [Pressure_Data,Temperature_Data]
    
    return(PT)
    

# LOCAL TO MSLP 

In [2]:

def TO_MSLP(Pres, Temp, elevation):
    '''
    Calculate Mean Sea Level Pressure (MSLP) using the Hypsometric equation.
    
    Args:
    - Pres: Pressure data (already corrected for temperature).
    - Temp: Temperature data in degrees Celsius.
    - elevation: Elevation in meters.
    
    Returns:
    - Updated pressure data with MSLP.
    '''
    
    A = elevation * 9.81 
    B = 287 * (Temp['degC'] + 273.15)
    C = A/B
    C = pd.to_numeric(C, errors='coerce')
    D = np.exp(C)
    MSLP = Pres['Correct_Pres_Temp'] * D
    return MSLP


def TO_SP(MSLP, Temp, elevation):
    '''
    Calculate Mean Sea Level Pressure (MSLP) using the Hypsometric equation.
    
    Args:
    - Pres: Pressure data (already corrected for temperature).
    - Temp: Temperature data in degrees Celsius.
    - elevation: Elevation in meters.
    
    Returns:
    - Updated pressure data with MSLP.
    '''
    
    A = elevation * 9.81 
    B = 287 * (Temp['degC'] + 273.15)
    C = A/B
    C = pd.to_numeric(C, errors='coerce')
    D = np.exp(C)

    SP = MSLP/D
    return SP

# RHTEST CONVERT DATA CONVERTER

In [4]:
#RHTest adjustments WE
def rh_fmt_row_month_mean_via_WMO_Guide(df_old,Raindays = False):
    """ This will transform which has a row per observing time into a format readable by the RH test software
    
    Args:
    df: dataframe with a row per observing time
    """
    
    columns = df_old.reset_index().columns #'Date','MSLP'

    #extract the monthlies under WMO guidelines 
    df = WMO_Guide_Months(df_old,columns[0],columns[1],Raindays)

    #fill any nans as -999.99
    df.fillna(-999.99,inplace=True)   

    #Extract year month and set day to 0
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = 0

    #Reset the index to remmove
    df = df.reset_index(drop=True)
    
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3] #reorder column order so year month day appear first
    df = df[cols]
    
    newdf = df[['year','month','day','MSLP']]
    
    return newdf

'''
The second RHTest format ensures any periods where only monthlies are avalaible can be used
'''

def rh_fmt_row_month_mean_all(df_old):
    """ This will transform which has a row per observing time into a format readable by the RH test software
    
    Args:
    df: dataframe with a row per observing time
    """
    df = df_old
    columns = df_old.columns #'MSLP'
    
    df = df.resample('MS').mean()

    #We create the full index for the daily data with the monthly data first full month and last month
    full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='MS')
    # Reindex the original DataFrame to this complete index
    df = df.reindex(full_index).round(1)
    df.fillna(-999.99,inplace=True)   

    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = 0
    
    df = df.reset_index(drop=True)
    
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3] #reorder column order so year month day appear first
    df = df[cols]
    
    newdf = df[['year','month','day','MSLP']]
    
    return newdf

'''
The third rhtest, is just the daily variant
'''
def rh_fmt_row_day_mean(df_old):
    """ This will transform which has a row per observing time into a format readable by the RH test software
    
    Args:
    df: dataframe with a row per observing time
    """
    
    df = df_old
    #We create the full index for the daily data with the monthly data first full month and last month
    full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    # Reindex the original DataFrame to this complete index
    df = df.reindex(full_index).round(1)
    df.fillna(-999.99,inplace=True)   

    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    
    df = df.reset_index(drop=True)
    
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3] #reorder column order so year month day appear first
    df = df[cols]
    
    newdf = df[['year','month','day','MSLP']]
    
    return newdf

# LINEAR REGRESSION EQUATIONS

In [32]:
def Linear_Regression_Equations(Trials, hours, Data):
    '''
    Parameters
    --------------
    Trials : Integer
        The number of trails you want to run the estimation training over.
        
    hours : array
    
    Data : DataFrame/Dictionary
        Using the observations and the training data we can have created a dictionary of DataFrames
        that have trialed that have been sampled by the lenght of the data avalaible for that month.
    
    Returns
    --------------
    Regressed_Trial : Dictionary/DataFrames
        For each trial, the set of equations for each sub-daily time, Trained DEs and Month will be generated for application 
        to estimate the DEs of the inputted data for estimation.
    '''
    
    #Create dictionaries
    Regressed_Trial = {}
    
    #Define the month names
    Month_Name = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

    #Being the for loop by extracting the month name
    for month_num in range(0,12):
        #Extract the month name_
        Month_Str =  Month_Name[month_num]
        #This is useful in the key as it is Aug_9_Mn_Samp where month_hour_mx/mn_samp
        
        #Now using the trials lets extract the trials within the particular dictionary
        for trial_number in range(1,Trials+1):
            #Now this is all the arrays that will be appended to at the end that include the linear
            #regression line components, A and B and the Correlation by the spearman r
            AMx_Total = []
            BMx_Total = []
            CORRMx_Total = []
            Time = []
            AMn_Total = []
            BMn_Total = []
            CORRMn_Total = []
            
            #Now for loop to extract the data and get the regression
            for i in hours:
                #---MAX---#
                #Extract the maximum data
                Mxt = Data.get('{}_{}_Mx_Samp'.format(Month_Str,i))
                #Get the linear formula and the correlation of the data
                AMx, BMx, corrMx = linear_regression_polyfit(Mxt['temp Run {}'.format(trial_number)],Mxt['Max Run {}'.format(trial_number)])
                #Append it all
                AMx_Total.append(AMx)
                BMx_Total.append(BMx)
                CORRMx_Total.append(corrMx)
                #Repeat for min
                #---MIN---#
                Mnt = Data.get('{}_{}_Mn_Samp'.format(Month_Str,i))
                AMn, BMn, corrMn = linear_regression_polyfit(Mnt['temp Run {}'.format(trial_number)],Mnt['Min Run {}'.format(trial_number)])
                Time.append(int(i)) 
                AMn_Total.append(AMn)
                BMn_Total.append(BMn)
                CORRMn_Total.append(corrMn)

            #Add it all into a dataframe
            Time = pd.Series(Time,name = 'Hours')
            
            AMX = pd.Series(AMx_Total,name = 'A')
            BMX = pd.Series(BMx_Total,name = 'B')
            corrMX = pd.Series(CORRMx_Total,name = 'Correlation')
            ItemsMX = pd.concat([Time,AMX,BMX,corrMX],axis = 1)
            
            AMN = pd.Series(AMn_Total,name = 'A')
            BMN = pd.Series(BMn_Total,name = 'B')
            corrMN = pd.Series(CORRMn_Total,name = 'Correlation')
            ItemsMN = pd.concat([Time,AMN,BMN,corrMN],axis = 1)
            
            Regressed_Trial["{}".format(Month_Str) + "_" + 'Trial'+ "_" + str(trial_number) + "_" + "Mx"] = ItemsMX
            Regressed_Trial["{}".format(Month_Str) + "_" + 'Trial'+ "_" + str(trial_number) + "_" + "Mn"] = ItemsMN
    return(Regressed_Trial)




#Now develop the linear regression equation
def linear_regression_polyfit(x,y):
    #Find the linear Relationship
    A, B = np.polyfit(x, y, 1)
    #Find the correlation                  
    corr, _ = spearmanr(x, y)
    return(A,B,corr)

# ZAK QM/PPM FUNCTION

In [33]:
#Lets get Zaks QM adjustment code from the data to esnure we can do this and correct the Perth Metro to Perth Airport
def qm_transfer(reference,uncorrected,value):
    #reference = vector of values representing the percentiles of the reference data
    #uncorrected = vector of values representing the percentiles of the data to be corrected
    #value = Value to be adjusted
    
    #  """Use source and destination percentiles to get the homogenised value.
    ## From Linden Ashcroft's R script (adapted from BoM Python script)
    #Given two lists of values representing, for example,
    #temperatures at the 5th to 95th percentiles, and a value
    #representing an amount to be homogenised, return the
    #homogenised value.
    #"""
    
    if pd.isna(value):
        return(np.nan)
    
    if value < round(uncorrected.iloc[0],3):
        return value + (reference.iloc[0] - uncorrected.iloc[0])
    
        #if the value is less than the lowest percentile, 
        #adjustment is difference between
        #the lowest percentile of the two
    
    if value > round(uncorrected.iloc[-1],3):
        return value + (reference.iloc[-1] - uncorrected.iloc[-1])
    
        #if the value is greater than the highest percentile, 
        #adjustment is difference between
        #the highest percentile of the two
    
    ndx = min(bisect_left(np.array(uncorrected), value), len(uncorrected) - 1)
    
    num_equal = np.count_nonzero(np.array(round(uncorrected,3) == value))
    
    if num_equal == 1:
        return value + (reference.iloc[ndx] - uncorrected.iloc[ndx])
        #If the value is an exact percentile value, 
        #then the adjustment is the difference between that percentile
    elif num_equal > 1:
        offset = random.randint(1, num_equal) - 1
        return value + (reference.iloc[ndx + offset] - uncorrected.iloc[ndx + offset])
    
        #If there are two percentiles with the value, 
        #pick one at random and use that to find the adjustment
        
    else:
        return (((reference.iloc[ndx] - reference.iloc[ndx-1])/
                   (uncorrected.iloc[ndx] - uncorrected.iloc[ndx-1]))*
                  (value - uncorrected.iloc[ndx-1]) + reference.iloc[ndx-1])
    
        #Otherwise, find the percentiles 'bin' that the value fits in, 
        #subtract the Glaisher percentile value from the value of interest
        #multiply that by a ratio of the two percentiles 
        #(if they are changing at the same rate this term will be 1) 
        #and add the Stevenson screen value for that percentile value
        
        #JR input, dont worry about the glaisher stuff, our focus is
        #on the pressure and this is something that can be similar

# COMBINING PERTHS DATASETS TOGETHER

In [34]:
def Perth_Combination(PM_MSLP,PA_MSLP,PRO_MSLP_B,PRO_MSLP,PO_MSLP,PG_MONTHLY,PG_MSLP,SWR_MSLP):
    ######################### PA TO PM
    
    #using Zaks code step by step Testing how 2 and 5 years impact the final product
    historical = PA_MSLP[['MSLP']].loc[:'1996-01-11']
    modern = PM_MSLP[['MSLP']].loc['1994-01-12':'1996-01-11']
    idx = historical.index.intersection(modern.index)
    modern_overlap = modern.loc[idx]
    historical_overlap = historical.loc[idx]
    
    timestamp = historical.columns
    correctedDf = historical.copy()
    for col in historical:
        historicalPct = historical_overlap[col].quantile(np.arange(0.05,1,0.05)) #calculate the percentiles for both datasets 
        modernPct = modern_overlap[col].quantile(np.arange(0.05,1,0.05))
        correctedList = []
        for j in range(len(historical[col])):
            correctedList.append(qm_transfer(modernPct,historicalPct,historical[col][j])) #loop through each value and correct
        correctedDf[col + '_corrected'] = correctedList
        del historicalPct, modernPct, correctedList
    corrected = correctedDf.drop(correctedDf.columns[0:len(historical.columns)], axis=1)
    corrected.columns = corrected.columns.str.replace('_corrected','')
    
    #Perth Metro reference
    PERTH_Pres = PM_MSLP[['MSLP']]
    
    #Add Perth Airport Corrected to PER_011_P_MSLP
    
    PERTH_Pres =  pd.concat([corrected[:'1994-01-11'],PERTH_Pres], axis =0)
    
    #########################
    #########################
    #########################
    ######################### PRO B TO PM
    #########################
    #########################
    #########################
    #########################

    #using Zaks code step by step Testing how 2 and 5 years impact the final product
    historical = PRO_MSLP_B[['MSLP']].loc[:'1992-04-10']
    modern = PERTH_Pres.loc['1990-04-11':'1992-04-10']
    idx = historical.index.intersection(modern.index)
    modern_overlap = modern.loc[idx]
    historical_overlap = historical.loc[idx]
    
    timestamp = historical.columns
    correctedDf = historical.copy()
    for col in historical:
        historicalPct = historical_overlap[col].quantile(np.arange(0.05,1,0.05)) #calculate the percentiles for both datasets 
        modernPct = modern_overlap[col].quantile(np.arange(0.05,1,0.05))
        correctedList = []
        for j in range(len(historical[col])):
            correctedList.append(qm_transfer(modernPct,historicalPct,historical[col][j])) #loop through each value and correct
        correctedDf[col + '_corrected'] = correctedList
        del historicalPct, modernPct, correctedList
    corrected = correctedDf.drop(correctedDf.columns[0:len(historical.columns)], axis=1)
    corrected.columns = corrected.columns.str.replace('_corrected','')
    
    #Add Perth Regional Office Corrected to PER_011_P_MSLP
    
    PERTH_Pres =  pd.concat([corrected,PERTH_Pres['1992-04-11':]], axis =0)
    
    #########################
    #########################
    #########################
    ######################### PRO U TO PM
    #########################
    #########################
    #########################
    #########################
    
    historical = PRO_MSLP.loc[:'1944-03-31']
    modern = PERTH_Pres.loc['1942-04-01':'1944-03-31']
    idx = historical.index.intersection(modern.index)
    modern_overlap = modern.loc[idx]
    historical_overlap = historical.loc[idx]
    
    timestamp = historical.columns
    correctedDf = historical.copy()
    for col in historical:
        historicalPct = historical_overlap[col].quantile(np.arange(0.05,1,0.05)) #calculate the percentiles for both datasets 
        modernPct = modern_overlap[col].quantile(np.arange(0.05,1,0.05))
        correctedList = []
        for j in range(len(historical[col])):
            correctedList.append(qm_transfer(modernPct,historicalPct,historical[col][j])) #loop through each value and correct
        correctedDf[col + '_corrected'] = correctedList
        del historicalPct, modernPct, correctedList
    corrected = correctedDf.drop(correctedDf.columns[0:len(historical.columns)], axis=1)
    corrected.columns = corrected.columns.str.replace('_corrected','')
    
    #Add Perth Regional Office Corrected to PERTH_Pres
    
    PERTH_Pres =  pd.concat([corrected[:'1942-03-31'],PERTH_Pres.loc['1942-04-01':]], axis =0)
    
    #########################
    #########################
    #########################
    ######################### PO TO PM
    #########################
    #########################
    #########################
    #########################
    
    #using Zaks code step by step Testing how 2 and 5 years impact the final product
    historical = PO_MSLP.sort_index().loc[:'1908-12-31']
    modern = PERTH_Pres.sort_index().loc['1907-01-01':'1908-12-31']
    idx = historical.index.intersection(modern.index)
    modern_overlap = modern.loc[idx]
    historical_overlap = historical.loc[idx]
    
    timestamp = historical.columns
    correctedDf = historical.copy()
    for col in historical:
        historicalPct = historical_overlap[col].quantile(np.arange(0.05,1,0.05)) #calculate the percentiles for both datasets 
        modernPct = modern_overlap[col].quantile(np.arange(0.05,1,0.05))
        correctedList = []
        for j in range(len(historical[col])):
            correctedList.append(qm_transfer(modernPct,historicalPct,historical[col][j])) #loop through each value and correct
        correctedDf[col + '_corrected'] = correctedList
        del historicalPct, modernPct, correctedList
    corrected = correctedDf.drop(correctedDf.columns[0:len(historical.columns)], axis=1)
    corrected.columns = corrected.columns.str.replace('_corrected','')
    
    #Add Perth Regional Office Corrected to PER_011_P_MSLP
    
    PERTH_Pres =  pd.concat([corrected[:'1906-12-31'],PERTH_Pres], axis =0)
    
    #########################
    #########################
    #########################
    ######################### PG TO PM
    #########################
    #########################
    #########################
    #########################
    
    #using Zaks code step by step Testing how 2 and 5 years impact the final product
    historical = PG_MSLP.sort_index().loc['1880':'1898-12-31']
    modern = PERTH_Pres.sort_index().loc['1897-01-01':'1898-12-31']
    idx = historical.index.intersection(modern.index)
    modern_overlap = modern.loc[idx]
    historical_overlap = historical.loc[idx]
    
    timestamp = historical.columns
    correctedDf = historical.copy()
    for col in historical:
        historicalPct = historical_overlap[col].quantile(np.arange(0.05,1,0.05)) #calculate the percentiles for both datasets 
        modernPct = modern_overlap[col].quantile(np.arange(0.05,1,0.05))
        correctedList = []
        for j in range(len(historical[col])):
            correctedList.append(qm_transfer(modernPct,historicalPct,historical[col][j])) #loop through each value and correct
        correctedDf[col + '_corrected'] = correctedList
        del historicalPct, modernPct, correctedList
    corrected = correctedDf.drop(correctedDf.columns[0:len(historical.columns)], axis=1)
    corrected.columns = corrected.columns.str.replace('_corrected','')
    
    #Add Perth Regional Office Corrected to PER_011_P_MSLP
    
    PERTH_Pres =  pd.concat([corrected[:'1896-12-31'],PERTH_Pres], axis =0)

    
    #########################
    #########################
    #########################
    ######################### PG Monthly TO PM
    #########################
    #########################
    #########################
    #########################
    

    #using Zaks code step by step Testing how 2 and 5 years impact the final product, this is using a monthly adjustment
    historical = PG_MONTHLY.sort_index().loc[:'1881-12']
    modern = PERTH_Pres.sort_index().loc['1880-01-01':'1881-12-31'].resample('D').mean().resample('MS').mean()
    idx = historical.index.intersection(modern.index)
    modern_overlap = modern.loc[idx]
    historical_overlap = historical.loc[idx]
    
    timestamp = historical.columns
    correctedDf = historical.copy()
    for col in historical:
        historicalPct = historical_overlap[col].quantile(np.arange(0.05,1,0.05)) #calculate the percentiles for both datasets 
        modernPct = modern_overlap[col].quantile(np.arange(0.05,1,0.05))
        correctedList = []
        for j in range(len(historical[col])):
            correctedList.append(qm_transfer(modernPct,historicalPct,historical[col][j])) #loop through each value and correct
        correctedDf[col + '_corrected'] = correctedList
        del historicalPct, modernPct, correctedList
    corrected = correctedDf.drop(correctedDf.columns[0:len(historical.columns)], axis=1)
    corrected.columns = corrected.columns.str.replace('_corrected','')
    
    #Add Perth Regional Office Corrected to PER_011_P_MSLP
    
    PERTH_Pres =  pd.concat([corrected[:'1879-12-31'],PERTH_Pres], axis =0)
    
    
    #########################
    #########################
    #########################
    ######################### SWR TO PM
    #########################
    #########################
    #########################
    #########################
    
    #Start by extracting data from that month only with the new monthly dataset, therefore the mean adjustment is from 
    PER_1880_1909 = PERTH_Pres.resample('D').mean().resample('MS').mean().loc['1876':'1905'].groupby([PERTH_Pres.resample('D').mean().resample('MS').mean().loc['1876':'1905'].index.month]).mean().reset_index().rename(columns = {'Date':'Month'}).set_index('Month')
    
    
    PER_1852_1875 = SWR_MSLP.loc['1852':'1875'].groupby([SWR_MSLP.loc['1852':'1875'].index.month]).mean().reset_index().rename(columns = {'Date':'Month'}).set_index('Month')
    
    
    PER_1830_1851 = SWR_MSLP.loc['1830':'1851'].groupby([SWR_MSLP.loc['1830':'1851'].index.month]).mean().reset_index().rename(columns = {'Date':'Month'}).set_index('Month')
    
    PER_1830_1875 = SWR_MSLP.loc['1830':'1875'].groupby([SWR_MSLP.loc['1830':'1875'].index.month]).mean().reset_index().rename(columns = {'Date':'Month'}).set_index('Month')
    
    #Get the difference
    PER_30_51_Dif = SWR_MSLP.loc['1830':'1851'].copy().reset_index()
    PER_30_51_Dif = PER_30_51_Dif.drop_duplicates(subset='Date', keep='last').set_index('Date')
    
    
    date = []
    pressure_dif = []
    for i in PER_30_51_Dif.index:
        #Extract month
        month = i.month
        
        if not np.isnan(PER_30_51_Dif.loc["{}".format(i)].values):
            date.append(i)
            pressure_dif.append(PER_30_51_Dif.loc["{}".format(i)].values[0]  - PER_1830_1851.loc[month].values[0])
        else:
            date.append(i)
            pressure_dif.append(np.nan)
            
    # Convert the list of timestamps to a pandas DataFrame
    date = pd.DataFrame({'Date': date})
    pressure_dif = pd.DataFrame({'MSLP_DIF': pressure_dif})
    
        
    PER_30_51_Dif = pd.concat([date,pressure_dif],axis = 1).set_index('Date')
    
    
    #########################################################
    #Now add that onto the average of the 1880-1909 plot
    date = []
    pressure_mean_adj = []
    for i in PER_30_51_Dif.index:
        #Extract month
        month = i.month
        
        if not np.isnan(PER_30_51_Dif.loc["{}".format(i)].values):
            date.append(i)
            pressure_mean_adj.append(PER_30_51_Dif.loc["{}".format(i)].values[0]  + PER_1880_1909.loc[month].values[0])
        else:
            date.append(i)
            pressure_mean_adj.append(np.nan)
            
    # Convert the list of timestamps to a pandas DataFrame
    date = pd.DataFrame({'Date': date})
    pressure_mean_adj = pd.DataFrame({'MSLP': pressure_mean_adj})
    
        
    PER_30_51_AJD = pd.concat([date,pressure_mean_adj],axis = 1).set_index('Date')
    
    
    
    #####################################################################################3333
    
    #Get the difference
    PER_52_75_Dif = SWR_MSLP.loc['1852':'1875'].copy().reset_index()
    PER_52_75_Dif = PER_52_75_Dif.drop_duplicates(subset='Date', keep='last').set_index('Date')
    date = []
    pressure_dif = []
    for i in PER_52_75_Dif.index:
        #Extract month
        month = i.month
        
        if not np.isnan(PER_52_75_Dif.loc["{}".format(i)].values):
            date.append(i)
            pressure_dif.append(PER_52_75_Dif.loc["{}".format(i)].values[0]  - PER_1852_1875.loc[month].values[0])
        else:
            date.append(i)
            pressure_dif.append(np.nan)
            
    # Convert the list of timestamps to a pandas DataFrame
    date = pd.DataFrame({'Date': date})
    pressure_dif = pd.DataFrame({'MSLP_DIF': pressure_dif})
    
        
    PER_52_75_Dif = pd.concat([date,pressure_dif],axis = 1).set_index('Date')



    
    #########################################################
    #Now add that onto the average of the 1880-1909 plot
    date = []
    pressure_mean_adj = []
    for i in PER_52_75_Dif.index:
        #Extract month
        month = i.month
        
        if not np.isnan(PER_52_75_Dif.loc["{}".format(i)].values):
            date.append(i)
            pressure_mean_adj.append(PER_52_75_Dif.loc["{}".format(i)].values[0]  + PER_1880_1909.loc[month].values[0])
        else:
            date.append(i)
            pressure_mean_adj.append(np.nan)
            
    # Convert the list of timestamps to a pandas DataFrame
    date = pd.DataFrame({'Date': date})
    pressure_mean_adj = pd.DataFrame({'MSLP': pressure_mean_adj})
    
        
    PER_52_75_AJD = pd.concat([date,pressure_mean_adj],axis = 1).set_index('Date')
    

    #####################################################################################3333
    
    #Get the difference
    PER_30_75_Dif = SWR_MSLP.loc['1830':'1875'].copy().reset_index()
    PER_30_75_Dif = PER_30_75_Dif.drop_duplicates(subset='Date', keep='last').set_index('Date')
    date = []
    pressure_dif = []
    for i in PER_30_75_Dif.index:
        #Extract month
        month = i.month
        
        if not np.isnan(PER_30_75_Dif.loc["{}".format(i)].values):
            date.append(i)
            pressure_dif.append(PER_30_75_Dif.loc["{}".format(i)].values[0]  - PER_1830_1875.loc[month].values[0])
        else:
            date.append(i)
            pressure_dif.append(np.nan)
            
    # Convert the list of timestamps to a pandas DataFrame
    date = pd.DataFrame({'Date': date})
    pressure_dif = pd.DataFrame({'MSLP_DIF': pressure_dif})
    
        
    PER_30_75_Dif = pd.concat([date,pressure_dif],axis = 1).set_index('Date')



    
    #########################################################
    #Now add that onto the average of the 1880-1909 plot
    date = []
    pressure_mean_adj = []
    for i in PER_30_75_Dif.index:
        #Extract month
        month = i.month
        
        if not np.isnan(PER_30_75_Dif.loc["{}".format(i)].values):
            date.append(i)
            pressure_mean_adj.append(PER_30_75_Dif.loc["{}".format(i)].values[0]  + PER_1880_1909.loc[month].values[0])
        else:
            date.append(i)
            pressure_mean_adj.append(np.nan)
            
    # Convert the list of timestamps to a pandas DataFrame
    date = pd.DataFrame({'Date': date})
    pressure_mean_adj = pd.DataFrame({'MSLP': pressure_mean_adj})
    


    
    PER_30_75_AJD = pd.concat([date,pressure_mean_adj],axis = 1).set_index('Date')

    
    
    '''
    ####################################################################
    PER_30_75_AJD= pd.concat([PER_30_51_AJD,PER_52_75_AJD],axis = 0)
    '''

    ##############################################################
    
    PER_30_24= pd.concat([PER_30_75_AJD,PERTH_Pres],axis = 0)
    return(PER_30_24)
    

# STORM FINDER

In [35]:
def Storm_Finder_AP_2009(Pressure,Raindays_Av,Raindays,Rainfall,Extreme,RD):
    
    #Fix data so its all filled up with 24hours
    All_Dates = pd.date_range(start=Pressure.index.min(), end=Pressure.index.max(), freq='h')
    All_Dates = pd.DataFrame(index=All_Dates, columns=[Pressure.columns[0]])
    
    #Insert the missing hours into the Pressure dataset
    Pressure = All_Dates.combine_first(Pressure)
    
    #Extract 9am and 3pm only
    Pressure = Pressure[Pressure.index.hour.isin([9, 15])]
    
    # Make sure Pressure is sorted by datetime index
    Pressure = Pressure.sort_index()
    
    # Calculate time difference to the next time (in hours)
    Pressure["Hours_to_next"] = (Pressure.index.to_series().shift(-1) - Pressure.index.to_series()).dt.total_seconds() / 3600
    
    # Calculate tendency (difference between current and next MSLP)
    Pressure["Tendency"] = Pressure[P_MSLP_SD.columns[0]].shift(-1) - Pressure[Pressure.columns[0]]
    #The tendency is calculated on the day of focos so row 1003.0 3.8, means in 6hr its 1006.8
    #This is to indentify the change on that particular day
    
    #Now we need to work out the 1st and 99th percentile tendency values of both 6hr and 18hr
    T_6hr = Pressure[Pressure['Hours_to_next'] == 6][['Tendency']]
    T_6hr["Tendency"] = pd.to_numeric(T_6hr["Tendency"], errors='coerce')
    
    T_18hr = Pressure[Pressure['Hours_to_next'] == 18][['Tendency']]
    T_18hr["Tendency"] = pd.to_numeric(T_18hr["Tendency"], errors='coerce')
    
    
    #Then work out 1st and 99th percentile for each month
    T_6hr['month'] = T_6hr.index.month
    T_18hr['month'] = T_18hr.index.month
    
    #For each month, now we have the range where the most extreme cases occur
    percentiles_ind_6hr = T_6hr.dropna().groupby('month')['Tendency'].quantile([0.01, 0.99])
    percentiles_ind_18hr = T_18hr.dropna().groupby('month')['Tendency'].quantile([0.01, 0.99])
    
    # Convert to DataFrame and unstack the quantile level
    percentiles_ind_6hr = percentiles_ind_6hr.unstack(level=1)
    percentiles_ind_18hr = percentiles_ind_18hr.unstack(level=1)
    
    # rename columns for clarity
    percentiles_ind_6hr.columns = ['1st', '99th']
    percentiles_ind_18hr.columns = ['1st', '99th']

    print(percentiles_ind_6hr)
    print(percentiles_ind_18hr)
    
    #6hr
    date_kept_6 = []
    
    #Iterate through the entire timeseries
    for row in T_6hr.itertuples():
        # Extract the month and tendency from the row
        mth = row.month
        tend = row.Tendency
    
        #Extract the 99th and 1st percentile
        if (tend <= percentiles_ind_6hr['1st'][mth]) | (tend >= percentiles_ind_6hr['99th'][mth]):
            date_kept_6.append(row.Index)
        else:
            x = 0
    
    #6hr
    date_kept_18 = []
    
    #Iterate through the entire timeseries
    for row in T_18hr.itertuples():
        # Extract the month and tendency from the row
        mth = row.month
        tend = row.Tendency
    
        #Extract the 99th and 1st percentile
        if (tend <= percentiles_ind_18hr['1st'][mth]) | (tend >= percentiles_ind_18hr['99th'][mth]):
            date_kept_18.append(row.Index)
        else:
            x = 0
    date_kept_6 = pd.DataFrame(date_kept_6, columns=['Date']).set_index('Date')
    date_kept_18 = pd.DataFrame(date_kept_18, columns=['Date']).set_index('Date')
    
    #Combine the potential storms together
    Pot_Storms = pd.concat([date_kept_6,date_kept_18],axis = 1).sort_index()
    #Merge with the actual tendency and MSLP data
    Pot_Storms = pd.merge(Pot_Storms,Pressure,how = 'inner',left_index = True, right_index= True)
    
    #Remove duplicated dates
    Pot_Storms['Date'] = Pot_Storms.index.strftime('%Y-%m-%d')
    Pot_Storms = Pot_Storms.drop_duplicates(subset='Date', keep='first')

    #ID the storms
    # Calculate the difference between consecutive dates
    Pot_Storms['Date'] = pd.to_datetime(Pot_Storms['Date'],dayfirst = True, format='mixed')
    Pot_Storms['Time Between Potential Storms'] = Pot_Storms['Date'].diff().dt.days
    # Identify new groups where the difference is not 1 day
    Pot_Storms['Storms'] = (Pot_Storms['Time Between Potential Storms'] != 1).cumsum()
    
    #Now add next periods pressure
    Pot_Storms.set_index('Date',inplace = True)
    Pot_Storms['MSLP Next Time'] = Pot_Storms['MSLP'] + Pot_Storms['Tendency']
    Wind_Storms_Included = Pot_Storms
    if Raindays_Av == True:
        #filter raindays into it
        Pot_Storms = pd.merge(Pot_Storms, Raindays, how='inner', left_index=True, right_index=True)
        Pot_Storms = pd.merge(Pot_Storms, Rainfall, how='inner', left_index=True, right_index=True)
    
        #Identify if rainfall and raindays is in there
        Pot_Storms = Pot_Storms[Pot_Storms[Raindays.columns[0]] == 1]
        
        #Find the time between storms again
        Pot_Storms['Time Between Potential Storms'] = Pot_Storms.index.diff().days

        #Indentify winter storms
        Pot_Storms = Pot_Storms[(Pot_Storms.index.month == 6) | (Pot_Storms.index.month == 7) | (Pot_Storms.index.month == 8)]

        # Identify new groups where the difference is not 1 day
        Pot_Storms['Storms'] = (Pot_Storms['Time Between Potential Storms'] != 1).cumsum()



    if Extreme == True:
        #Identify storms with the most extreme tendencys 0.01 and 0.99
        E_percentiles_ind_6hr = T_6hr.dropna().groupby('month')['Tendency'].quantile([0.001, 0.999])
        E_percentiles_ind_18hr = T_18hr.dropna().groupby('month')['Tendency'].quantile([0.001, 0.999])
        # Convert to DataFrame and unstack the quantile level
        E_percentiles_ind_6hr = E_percentiles_ind_6hr.unstack(level=1)
        E_percentiles_ind_18hr = E_percentiles_ind_18hr.unstack(level=1)
        # rename columns for clarity
        E_percentiles_ind_6hr.columns = ['0.1th', '99.9th']
        E_percentiles_ind_18hr.columns = ['0.1th', '99.9th']

        #To find some of the biggest pressure jumps without rainfall included
        extreme_storms_BF_RF = []       
        Pot_Storms_BF_RF = Pot_Storms.loc[:'1879'].copy().reset_index()
        for i in range(len(Pot_Storms_BF_RF)):
            Data = Pot_Storms_BF_RF.iloc[i]
            Date = Data['Date']
            Month = Date.month
            Tend = Data['Tendency']
            Tendency_Window = Data['Hours_to_next']
            if (Tendency_Window == 6):
               if (Tend <= E_percentiles_ind_6hr['0.1th'][Month]) | (Tend >= E_percentiles_ind_6hr['99.9th'][Month]):
                   extreme_storms_BF_RF.append(Data)
               else:
                   pass
            elif (Tendency_Window == 18):
                if (Tend <= E_percentiles_ind_18hr['0.1th'][Month]) | (Tend >= E_percentiles_ind_18hr['99.9th'][Month]):
                    extreme_storms_BF_RF.append(Data)
                else:
                    pass

        extreme_storms_BF_RF = pd.DataFrame(extreme_storms_BF_RF) #in the 1800s

        extreme_storms_AF_RF =Pot_Storms[Pot_Storms[P_RF.columns[0]] >= 30].loc['1880':]
    
        extreme_storms = pd.concat([extreme_storms_BF_RF.set_index('Date'),extreme_storms_AF_RF], axis = 0).sort_index()
        extreme_storms['Time Between Potential Storms'] = extreme_storms.index.diff().days
        extreme_storms['Storms'] = (extreme_storms['Time Between Potential Storms'] != 1).cumsum()

        es_cols = extreme_storms.columns
    
        extreme_storms =extreme_storms.rename(columns = {es_cols[5] :'Next MSLP Reading (hPa)'})
        extreme_storms =extreme_storms.rename(columns = {es_cols[1] :'Hours To Next MSLP Reading'})
        extreme_storms =extreme_storms.rename(columns = {P_MSLP_SD.columns[0] :'MSLP (hPa)'})
        extreme_storms =extreme_storms.rename(columns = {es_cols[2] :'MSLP Change (hPa)'})
        extreme_storms =extreme_storms.rename(columns = {es_cols[7] :'Precipitation (mm)'})
        extreme_storms =extreme_storms.rename(columns = {es_cols[4] :'Storm ID'})

        del extreme_storms[es_cols[6]]
        extreme_storms = extreme_storms.reset_index().set_index('Storm ID')
        extreme_storms = extreme_storms[['Date','MSLP (hPa)', 'Precipitation (mm)', 'Hours To Next MSLP Reading', 'Next MSLP Reading (hPa)', 'MSLP Change (hPa)']] 
        extreme_storms['Hours To Next MSLP Reading'] =extreme_storms['Hours To Next MSLP Reading'].round(0)
        return(extreme_storms)

    if RD == False:
        Wind_Storms_Included = Wind_Storms_Included[(Wind_Storms_Included.index.month == 6) | (Wind_Storms_Included.index.month == 7) | (Wind_Storms_Included.index.month == 8)]
        # Identify new groups where the difference is not 1 day
        Wind_Storms_Included['Time Between Potential Storms'] = Wind_Storms_Included.index.diff().days
        Wind_Storms_Included['Storms'] = (Wind_Storms_Included['Time Between Potential Storms'] != 1).cumsum()
        return(Wind_Storms_Included)
    else:
        return(Pot_Storms)

# STORMS

In [2]:
def STORM_MSLP_ONLY(Events_Vec, Dataset,Titles):
    #Lets play with the predata
    # Shape of Australia
    borders_gdf = gpd.read_file(r"C:\Users\jarra\Desktop\RA ASSISTANT\FINALISED\Shape Files\World Shape\ne_10m_admin_0_countries.shp")
    
    # If needed, set the crs (coordinate reference system) for the GeoDataFrame
    # For example, if your data is in WGS84 (EPSG:4326), you can set the crs as follows:
    borders_gdf.crs = 'EPSG:4326'

    dot = 4
    #Find the length of the events
    length_events = len(Events_Vec)
    #Lets create a subplot
    fig, ax = plt.subplots(nrows=length_events, ncols=4, figsize=(21,3.2*length_events))
    
    #Lets make sure our borders are done correctly
    lat_min, lat_max = Dataset['lat'].min().item(), Dataset['lat'].max().item()
    lon_min, lon_max = Dataset['lon'].min().item(), Dataset['lon'].max().item()

    #Now lets create the anomaly for overall
    #Mean_Pressure_81_10 = Dataset.sel(time=slice('1981', '2010')).mean(dim='time')
    
    #For Winter Storms
    Mean_Pressure_81_10 = Dataset.sel(time=slice('1961', '1990'))
    Mean_Pressure_81_10 = Mean_Pressure_81_10.sel(time=Mean_Pressure_81_10['time'].dt.month.isin([6, 7, 8])).mean(dim='time')

    Pressure_Anomaly = Dataset - Mean_Pressure_81_10  
    
    #Colours 
    colorbar = 'RdBu_r'
    
    #Range
    P_range = np.arange(-16, 16 + 2, 2)

    
    #Now we have to create the for loop that produces the figure
    for i, event_date in enumerate(Events_Vec):
        #Day Of
        DOE = datetime.strptime(event_date, '%Y-%m-%d')
        
        #Day Before
        DBE = DOE - timedelta(days=1)

        #Day After
        DAE = DOE + timedelta(days=1)
        
        # Set the axes for the current row
        row_axes = ax[i]
        
        # Plot for Day Before
        P_cont = Pressure_Anomaly.prmsl.sel(time=DBE).plot.contourf(ax=row_axes[0], cmap=colorbar, levels=P_range, add_colorbar=False)
        DBE_YMD = DBE.strftime('%Y-%m-%d')
        row_axes[0].set_title('{}'.format(DBE_YMD),fontsize = 12)
        # Plot for Day Of
        P_cont = Pressure_Anomaly.prmsl.sel(time=DOE).plot.contourf(ax=row_axes[1], cmap=colorbar, levels=P_range, add_colorbar=False)
        DOE_YMD = DOE.strftime('%Y-%m-%d')
        row_axes[1].set_title('{}'.format(DOE_YMD),fontsize = 12)

        # Plot for Day After
        P_cont = Pressure_Anomaly.prmsl.sel(time=DAE).plot.contourf(ax=row_axes[2], cmap=colorbar, levels=P_range, add_colorbar=False)
        DAE_YMD = DAE.strftime('%Y-%m-%d')
        row_axes[2].set_title('{}'.format(DAE_YMD),fontsize = 12)



        for j in [0,1,2]:
            #Plot Australian Border
            Border = borders_gdf.plot(ax=row_axes[j], alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)
            #Set the plot borders
            row_axes[j].set_xlim(lon_min, lon_max)
            row_axes[j].set_ylim(lat_min, lat_max)
            #Plot Perth
            Perth = row_axes[j].plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')

            row_axes[j].set_xlabel('')
            row_axes[j].set_ylabel('')
            # Remove tick labels on the x and y axes
            row_axes[j].set_xticklabels([])
            row_axes[j].set_yticklabels([])
            row_axes[j].tick_params(axis='both', which='both', bottom=False, left=False)

        ax[i,0].set_ylabel('{}'.format(Titles[i]),fontsize = 12)

    #Fix the final column
    ax15 = ax[:,3]
    cbar_PM = fig.colorbar(P_cont, ax=ax15, location='left', shrink=0.5)
    # Set the font size of colorbar ticks
    cbar_PM.ax.tick_params(labelsize=14)
    # Create a text object for the label
    cbar_label = fig.text(0.77, 0.5, 'Mean Sea Level\nPressure Anomaly (hPa)', rotation=90, va='center', fontsize=18)

    for i in range(length_events):
        # Remove the last subplot in the rows
        fig.delaxes(ax[i,3])
        
        


    return fig, ax



# Example usage
#fig, ax = STORM__MSLP_ONLY(['1951-07-16', '1866-07-28','1972-02-10','1907-06-20','1915-01-25'], Daily_Pressure,['Storm','Storm','Tropical\nCyclone','Storm','Tropical\nCyclone'])



In [7]:
def STORM_V2(Year, date, hour ,date_for_xlim, PresTS, TempTS, Type):
    #Lets play with the predata
    # Shape of Australia
    borders_gdf = gpd.read_file(r"C:\Users\jarra\Desktop\RA ASSISTANT\FINALISED\Shape Files\World Shape\ne_10m_admin_0_countries.shp")
    
    # If needed, set the crs (coordinate reference system) for the GeoDataFrame
    # For example, if your data is in WGS84 (EPSG:4326), you can set the crs as follows:
    borders_gdf.crs = 'EPSG:4326'

    #20CR Pressure
    T2M =  xr.open_dataset(r"C:\Users\jarra\Desktop\RA ASSISTANT\DATA\Data\20CR\FOR EXTREME EVENT EXAMPLES\air.2m.{}.nc".format(Year))
    APCP =  xr.open_dataset(r"C:\Users\jarra\Desktop\RA ASSISTANT\DATA\Data\20CR\FOR EXTREME EVENT EXAMPLES\apcp.{}.nc".format(Year))
    MSLP =  xr.open_dataset(r"C:\Users\jarra\Desktop\RA ASSISTANT\DATA\Data\20CR\FOR EXTREME EVENT EXAMPLES\prmsl.{}.nc".format(Year))
    UWIND =  xr.open_dataset(r"C:\Users\jarra\Desktop\RA ASSISTANT\DATA\Data\20CR\FOR EXTREME EVENT EXAMPLES\uwnd.10m.{}.nc".format(Year))
    VWIND =  xr.open_dataset(r"C:\Users\jarra\Desktop\RA ASSISTANT\DATA\Data\20CR\FOR EXTREME EVENT EXAMPLES\vwnd.10m.{}.nc".format(Year))

    #Add 8hour to UTC to get it into AWST
    T2M = T2M.assign_coords(time=T2M.time + np.timedelta64(8, 'h'))
    APCP = APCP.assign_coords(time=APCP.time + np.timedelta64(8, 'h'))
    MSLP = MSLP.assign_coords(time=MSLP.time + np.timedelta64(8, 'h'))
    UWIND = UWIND.assign_coords(time=UWIND.time + np.timedelta64(8, 'h'))
    VWIND = VWIND.assign_coords(time=VWIND.time + np.timedelta64(8, 'h'))

    #Convert to degC
    T2M = T2M.air -273.15

    #Convert to hPa
    MSLP = MSLP.prmsl*0.01

    #Back to dataset
    MSLP= MSLP.to_dataset()
    T2M = T2M.to_dataset()
    APCP = APCP
    ################DATES
    
    
    #Spatial Series timeline of event
    Storm_1851_P20CR = MSLP.sel(time=slice(date_for_xlim[0], date_for_xlim[1]))
    Storm_1851_T20CR = T2M.sel(time=slice(date_for_xlim[0], date_for_xlim[1]))
    Storm_1851_Uwnd20CR = UWIND.sel(time=slice(date_for_xlim[0], date_for_xlim[1]))
    Storm_1851_Vwnd20CR = VWIND.sel(time=slice(date_for_xlim[0], date_for_xlim[1]))
    Storm_1851_APCP20CR = APCP.sel(time=slice(date_for_xlim[0], date_for_xlim[1]))


    #COMBINE U AND V WIND
    Storm_1851_Wnd20CR = np.sqrt(Storm_1851_Uwnd20CR.uwnd**2 + Storm_1851_Vwnd20CR.vwnd**2)
    Storm_1851_Wnd20CR = Storm_1851_Wnd20CR.to_dataset(name='wnd_spd')

    P20CR = Storm_1851_P20CR
    T20CR =  Storm_1851_T20CR
    WND20CR = Storm_1851_Wnd20CR
    APCP20CR = Storm_1851_APCP20CR

  
    TempTS =  TempTS.loc[date_for_xlim[0]: date_for_xlim[1]]
    PresTS =  PresTS.loc[date_for_xlim[0]: date_for_xlim[1]]

    
    
    
    
    
    
    
    
    #############################################################
    '''
    VARIABLES
    '''
    ###FOR LABELS###
    #Fontsize for title
    TL = 26
    #Fontsize for x and y 
    XYL = 16
    #Size of Perth
    dot = 8
    
    
    # Convert date strings to datetime objects
    date_for_xlim = [datetime.strptime(date_str, '%Y-%m-%d') for date_str in date_for_xlim]
    
    #Date of Impact 
    Event = date
    # Date of Event (DOE)
    DOE = datetime.strptime(date, '%Y-%m-%d')
    # Adjust the hour of the event
    DOE = DOE.replace(hour=hour)
    
    # Calculate the day before and after the event
    DBE = DOE - timedelta(hours=3)
    DAE = DOE + timedelta(hours=3)
    
    #Lat and Lon Coords
    lat_min, lat_max = -43, -25
    lon_min, lon_max = 100,  130

    #Colours 
    TM_C = 'coolwarm'
    PM_C = 'PRGn'
    RM_C = 'Blues'
    WM_C = 'Reds'





    #Get the datasetts to the respective region
    # Define the latitude and longitude bounds
    lon_slice = slice(lon_min, lon_max)
    lat_slice = slice(lat_min, lat_max)

    # Select the desired geographical area from the dataset
    P_subset = P20CR.sel(lon=lon_slice, lat=lat_slice)    
    T_subset = T20CR.sel(lon=lon_slice, lat=lat_slice)    
    R_subset = APCP20CR.sel(lon=lon_slice, lat=lat_slice)    
    W_subset = WND20CR.sel(lon=lon_slice, lat=lat_slice)    
    
    #Ranges of Data, so Spread, and mean
    PMean = [np.floor(P_subset.prmsl.min().values.item()),np.ceil(P_subset.prmsl.max().values.item())]
    TMean = [np.floor(T_subset.air.min().values.item()),np.ceil(T_subset.air.max().values.item())]
    RMean = [np.floor(R_subset.apcp.min().values.item()),np.ceil(R_subset.apcp.max().values.item())]
    WMean = [np.floor(W_subset.wnd_spd.min().values.item()),np.ceil(W_subset.wnd_spd.max().values.item())]
    # Set contour levels as integer values
    TM_levels = np.arange(TMean[0], TMean[1] + 1,1)
    PM_levels = np.arange(PMean[0], PMean[1] + 1,1)
    RM_levels = np.arange(0, RMean[1] + 1,0.2)
    WM_levels = np.arange(0, WMean[1] + 1,1)
    
    
    
    # Start the plot
    fig = plt.figure(figsize=(26, 24))
    #gs = fig.add_gridspec(5, 4)
    gs = fig.add_gridspec(5, 4, height_ratios=[1, 1.5, 1, 1, 1])

    ##############################################################
    '''
    TIMESERIES (TEMPERATURE AND PRESSURE)
    '''

    # Create a twinax plot for Temp and Pressure from the timeseries
    
    # Plot Pressure
    ax1 = fig.add_subplot(gs[0, 0:3])
    ax1.plot(PresTS, color='red', marker='x', linestyle='-', markerfacecolor='red')

    # Temperature
    ax2 = ax1.twinx()
    ax2.plot(TempTS, color='blue', marker='o', linestyle='--', markerfacecolor='blue')

    # Set x-axis ticks and labels using matplotlib.dates
    locator = mdates.DayLocator(interval=1)  # Interval of 2 days

    ax1.xaxis.set_major_locator(mdates.DayLocator(interval=1))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))  # Adjust date format as needed
    # Show all vertical grid lines
    ax1.xaxis.grid(which='both', linestyle='-', linewidth=0.5)
    # Set x-axis label and adjust fontsize
    ax1.yaxis.grid(which='both', linestyle='-', linewidth=0.5)
    # Adjust fontsize of x-axis tick labels
    ax1.tick_params(axis='x', labelsize=12, rotation=45)  # Adjust labelsize as needed
    ax1.tick_params(axis='y',labelsize=12)
    
    # Set y-axis tick colors to match the lines
    ax1.tick_params(axis='y', colors='red')
    ax2.tick_params(axis='y', colors='blue')
    
    
    
    
    ##############################################################
    '''
    SPATIAL PLOT (PRESSURE)
    '''
    #PRESSURE
    ax3 = fig.add_subplot(gs[1, 0])

    ax = ax3
    #Plot by contours
    PM_contour = P_subset.prmsl.sel(time=DBE).plot.contourf(ax=ax, cmap=PM_C, levels=PM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)
    
    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #PRESSURE
    ax4 = fig.add_subplot(gs[1, 1])

    ax = ax4
    #Plot by contours
    P_subset.prmsl.sel(time=DOE).plot.contourf(ax=ax, cmap=PM_C, levels=PM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #PRESSURE
    ax5 = fig.add_subplot(gs[1, 2])

    ax = ax5
    #Plot by contours
    P_subset.prmsl.sel(time=DAE).plot.contourf(ax=ax, cmap=PM_C, levels=PM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    ##############################################################
    '''
    SPATIAL PLOT (TEMPERATURE)
    '''
    #TEMPERATURE
    ax6 = fig.add_subplot(gs[2, 0])

    ax = ax6
    #Plot by contours
    TM_contour = T_subset.air.sel(time=DBE).plot.contourf(ax=ax, cmap=TM_C, levels=TM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #TEMPERATURE
    ax7 = fig.add_subplot(gs[2, 1])

    ax = ax7
    #Plot by contours
    T_subset.air.sel(time=DOE).plot.contourf(ax=ax, cmap=TM_C, levels=TM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #TEMPERATURE
    ax8 = fig.add_subplot(gs[2, 2])

    ax = ax8
    #Plot by contours
    T_subset.air.sel(time=DAE).plot.contourf(ax=ax, cmap=TM_C, levels=TM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    ##############################################################
    '''
    SPATIAL PLOT (RAINFALL)
    '''
    
    #Rainfall
    ax9 = fig.add_subplot(gs[3, 0])

    ax = ax9
    #Plot by contours
    RM_contour = R_subset.apcp.sel(time=DBE).plot.contourf(ax=ax, cmap=RM_C, levels=RM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #Rainfall
    ax10 = fig.add_subplot(gs[3, 1])

    ax = ax10
    #Plot by contours
    R_subset.apcp.sel(time=DOE).plot.contourf(ax=ax, cmap=RM_C, levels=RM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #Rainfall
    ax11 = fig.add_subplot(gs[3, 2])

    ax = ax11
    #Plot by contours
    R_subset.apcp.sel(time=DAE).plot.contourf(ax=ax, cmap=RM_C, levels=RM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    
    ##############################################################
    '''
    SPATIAL PLOT (WINDS)
    '''
    #WIND
    ax12 = fig.add_subplot(gs[4, 0])

    ax = ax12
    #Plot by contours
    WM_contour = W_subset.wnd_spd.sel(time=DBE).plot.contourf(ax=ax, cmap=WM_C, levels=WM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #WIND
    ax13 = fig.add_subplot(gs[4, 1])

    ax = ax13
    #Plot by contours
    W_subset.wnd_spd.sel(time=DOE).plot.contourf(ax=ax, cmap=WM_C, levels=WM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    #WIND
    ax14 = fig.add_subplot(gs[4, 2])

    ax = ax14
    #Plot by contours
    W_subset.wnd_spd.sel(time=DAE).plot.contourf(ax=ax, cmap=WM_C, levels=WM_levels, add_colorbar=False)

    #Add the Australian Border
    Border = borders_gdf.plot(ax=ax, alpha=0.5, edgecolor='black', facecolor='none', linewidth=1)

    #Set the plot borders
    ax.set_xlim(lon_min, lon_max)
    ax.set_ylim(lat_min, lat_max)

    #Add Perth
    Perth = ax.plot(115.8605, -31.9505, 'ko', markersize=dot, label='Perth')
    ax.set_title('')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    
    ax15 = fig.add_subplot(gs[1, 3])
    cbar_PM = fig.colorbar(PM_contour, ax=ax15, location='left')
    ax16 = fig.add_subplot(gs[2, 3])
    cbar_TM = fig.colorbar(TM_contour, ax=ax16, location='left')
    ax17 = fig.add_subplot(gs[3, 3])
    cbar_RM = fig.colorbar(RM_contour, ax=ax17, location='left')
    ax18 = fig.add_subplot(gs[4, 3])
    cbar_WM = fig.colorbar(WM_contour, ax=ax18, location='left')
    # Removing empty subplots
    fig.delaxes(ax15)
    fig.delaxes(ax16)
    fig.delaxes(ax17)
    fig.delaxes(ax18)
    
    
    #tITLE ETC
    ax1.set_title('Pressure and Temperature Timeseries',fontsize = TL)
    ax1.set_xlabel('Date',fontsize = XYL)
    ax1.set_ylabel('MSLP (hPa)',color = 'red',fontsize = XYL)
    ax2.set_ylabel('Temperature ($^\circ$C)',color = 'blue',fontsize = XYL)
    
    #tITLE ETC
    ax4.set_title('20th Century Reanalysis Data',fontsize = TL)
    ax3.set_ylabel('3-Hourly Mean Sea Level\nPressure (hPa)',fontsize = XYL)
    ax6.set_ylabel('3-Hourly Temperature\nat 2m ($^\circ$C)',fontsize = XYL)
    ax9.set_ylabel('3-Hourly Accumulation\nPrecipitation (mm)',fontsize = XYL)
    ax12.set_ylabel('3-Hourly Wind\nSpeed at 10m (m/s)',fontsize = XYL)
    ax12.set_xlabel('{}-{}-{} {}:00'.format(DBE.day,DBE.month,DBE.year,DBE.hour),fontsize = XYL)
    ax13.set_xlabel('{}-{}-{} {}:00'.format(DOE.day,DOE.month,DOE.year,DOE.hour),fontsize = XYL)
    ax14.set_xlabel('{}-{}-{} {}:00'.format(DAE.day,DAE.month,DAE.year,DAE.hour),fontsize = XYL)
    plt.suptitle('{} {}'.format(Year, Type), fontsize = 28, y = 0.92, x = 0.42)

# word save table

In [1]:
#Save potential storms
def save_to_word(data,output):
    data = data.reset_index()
    data.replace(np.nan, 'N/A', inplace=True)
    
    # Convert Date to string format
    data['Date'] = data['Date'].dt.strftime('%d-%m-%Y')
    
    # Helper function to set font and center-align text
    def set_cell_font(cell, font_name="Times New Roman", font_size=10):
        for paragraph in cell.paragraphs:
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            for run in paragraph.runs:
                run.font.name = font_name
                run.font.size = Pt(font_size)
                # Ensure font is correctly set in XML
                r = run._element
                rPr = r.get_or_add_rPr()
                rFonts = rPr.find(qn('w:rFonts'))
                if rFonts is None:
                    rFonts = OxmlElement('w:rFonts')
                    rPr.append(rFonts)
                rFonts.set(qn('w:ascii'), font_name)
                rFonts.set(qn('w:hAnsi'), font_name)
            # Create Word document
    doc = Document()
    doc.add_heading('Extreme Storms Table', level=1)
    
    # Create table
    table = doc.add_table(rows=1, cols=len(data.columns))
    table.style = 'Table Grid'
    
    # Add headers
    hdr_cells = table.rows[0].cells
    for i, col in enumerate(data.columns):
        hdr_cells[i].text = str(col)
        set_cell_font(hdr_cells[i])
    
    # Add data rows
    for _, row in data.iterrows():
        row_cells = table.add_row().cells
        for i, item in enumerate(row):
            if isinstance(item, (float, int)):
                row_cells[i].text = str(round(item, 1))
            else:
                row_cells[i].text = str(item)
            set_cell_font(row_cells[i])
    
    doc.save(output)

In [ ]:
# OLD STORM FUNCTION

In [1]:
def Storm_Finder_AP_2009(Pressure,Months_Inc, Tendency = False):

    '''
    Pressure - Subdaily pressure record

    Output 
    Potential Storms 1 to 99
    Potential Pressure Extreme Storms 0.1 to 99.9
    '''
    #Fix data so its all filled up with 24hours
    All_Dates = pd.date_range(start=Pressure.index.min(), end=Pressure.index.max(), freq='h')
    All_Dates = pd.DataFrame(index=All_Dates, columns=[Pressure.columns[0]])
    
    #Insert the missing hours into the Pressure dataset
    Pressure = All_Dates.combine_first(Pressure)
    
    #Extract 9am and 3pm only
    Pressure = Pressure[Pressure.index.hour.isin([9, 15])]
    
    # Make sure Pressure is sorted by datetime index
    Pressure = Pressure.sort_index()
    
    # Calculate time difference to the next time (in hours)
    Pressure["Hours_to_next"] = (Pressure.index.to_series().shift(-1) - Pressure.index.to_series()).dt.total_seconds() / 3600
    
    # Calculate tendency (difference between current and next MSLP)
    Pressure["Tendency"] = Pressure[P_MSLP_SD.columns[0]].shift(-1) - Pressure[Pressure.columns[0]]
    #The tendency is calculated on the day of focos so row 1003.0 3.8, means in 6hr its 1006.8
    #This is to indentify the change on that particular day
    
    #Now we need to work out the 1st and 99th percentile tendency values of both 6hr and 18hr
    T_6hr = Pressure[Pressure['Hours_to_next'] == 6][['Tendency']]
    T_6hr["Tendency"] = pd.to_numeric(T_6hr["Tendency"], errors='coerce')
    
    T_18hr = Pressure[Pressure['Hours_to_next'] == 18][['Tendency']]
    T_18hr["Tendency"] = pd.to_numeric(T_18hr["Tendency"], errors='coerce')
    
    
    #Then work out 1st and 99th percentile for each month
    T_6hr['month'] = T_6hr.index.month
    T_18hr['month'] = T_18hr.index.month
    
    #For each month, now we have the range where the most extreme cases occur
    percentiles_ind_6hr = T_6hr.dropna().groupby('month')['Tendency'].quantile([0.01, 0.99])
    percentiles_ind_18hr = T_18hr.dropna().groupby('month')['Tendency'].quantile([0.01, 0.99])
    
    # Convert to DataFrame and unstack the quantile level
    percentiles_ind_6hr = percentiles_ind_6hr.unstack(level=1)
    percentiles_ind_18hr = percentiles_ind_18hr.unstack(level=1)
    
    # rename columns for clarity
    percentiles_ind_6hr.columns = ['1st', '99th']
    percentiles_ind_18hr.columns = ['1st', '99th']

    #6hr
    date_kept_6 = []
    
    #Iterate through the entire timeseries
    for row in T_6hr.itertuples():
        # Extract the month and tendency from the row
        mth = row.month
        tend = row.Tendency
    
        #Extract the 99th and 1st percentile
        if (tend <= percentiles_ind_6hr['1st'][mth]) | (tend >= percentiles_ind_6hr['99th'][mth]):
            date_kept_6.append(row.Index)
        else:
            x = 0
    
    #6hr
    date_kept_18 = []
    
    #Iterate through the entire timeseries
    for row in T_18hr.itertuples():
        # Extract the month and tendency from the row
        mth = row.month
        tend = row.Tendency
    
        #Extract the 99th and 1st percentile
        if (tend <= percentiles_ind_18hr['1st'][mth]) | (tend >= percentiles_ind_18hr['99th'][mth]):
            date_kept_18.append(row.Index)
        else:
            x = 0
    #Keeps date and hour
    date_kept_6 = pd.DataFrame(date_kept_6, columns=['Date']).set_index('Date')
    date_kept_18 = pd.DataFrame(date_kept_18, columns=['Date']).set_index('Date')

    #Combine the potential storms together
    Pot_Storms = pd.concat([date_kept_6,date_kept_18],axis = 1).sort_index()
    #Merge with the actual tendency and MSLP data
    Pot_Storms = pd.merge(Pot_Storms,Pressure,how = 'inner',left_index = True, right_index= True)
    
    #Remove duplicated dates
    Pot_Storms['Date'] = Pot_Storms.index.strftime('%Y-%m-%d')
    #Pot_Storms = Pot_Storms.drop_duplicates(subset='Date', keep='first')

    #ID the storms
    # Calculate the difference between consecutive dates
    Pot_Storms['Date'] = pd.to_datetime(Pot_Storms['Date'],dayfirst = True, format='mixed')
    Pot_Storms['Time Between Potential Storms'] = Pot_Storms['Date'].diff().dt.days
    # Identify new groups where the difference is not 1 day
    Pot_Storms['Storms'] = (Pot_Storms['Time Between Potential Storms'] > 1).cumsum() + 1
    
    #Now add next periods pressure
    Pot_Storms.set_index('Date',inplace = True)
    Pot_Storms['MSLP Next Time'] = Pot_Storms['MSLP'] + Pot_Storms['Tendency']
    Wind_Storms_Included = Pot_Storms #During winter
    
    Winter_Storms = ChatJR.filter_dataframe_by_months(Wind_Storms_Included, Months_Inc)
    
    # Calculate the difference between consecutive dates
    Winter_Storms['Time Between Potential Storms'] = Winter_Storms.index.diff().days
    # Identify new groups where the difference is not 1 day
    Winter_Storms['Storms'] = (Winter_Storms['Time Between Potential Storms'] > 1).cumsum()+1
    es_cols = Winter_Storms.columns
    
    Winter_Storms =Winter_Storms.rename(columns = {P_MSLP_SD.columns[0] :'MSLP (hPa)'})
    Winter_Storms =Winter_Storms.rename(columns = {es_cols[1] :'Hours To Next MSLP Reading'})
    Winter_Storms =Winter_Storms.rename(columns = {es_cols[2] :'MSLP Change (hPa)'})
    Winter_Storms =Winter_Storms.rename(columns = {es_cols[3] :'Days between Storms'})
    Winter_Storms =Winter_Storms.rename(columns = {es_cols[4] :'Storm ID'})
    Winter_Storms =Winter_Storms.rename(columns = {es_cols[5] :'Next MSLP Reading (hPa)'})
    Winter_Storms = Winter_Storms.reset_index().set_index('Storm ID')


    
    
    #Now lets get extreme winter storms based on pressure
    #Identify storms with the most extreme tendencys 0.01 and 0.99
    E_percentiles_ind_6hr = T_6hr.dropna().groupby('month')['Tendency'].quantile([0.001, 0.999])
    E_percentiles_ind_18hr = T_18hr.dropna().groupby('month')['Tendency'].quantile([0.001, 0.999])
    # Convert to DataFrame and unstack the quantile level
    E_percentiles_ind_6hr = E_percentiles_ind_6hr.unstack(level=1)
    E_percentiles_ind_18hr = E_percentiles_ind_18hr.unstack(level=1)
    # rename columns for clarity
    E_percentiles_ind_6hr.columns = ['0.1th', '99.9th']
    E_percentiles_ind_18hr.columns = ['0.1th', '99.9th']

    #To find some of the biggest pressure jumps without rainfall included
    extreme_storms_BF_RF = []       
    Pot_Storms_BF_RF = Pot_Storms.copy().reset_index()
    for i in range(len(Pot_Storms_BF_RF)):
        Data = Pot_Storms_BF_RF.iloc[i]
        Date = Data['Date']
        Month = Date.month
        Tend = Data['Tendency']
        Tendency_Window = Data['Hours_to_next']
        if (Tendency_Window == 6):
           if (Tend <= E_percentiles_ind_6hr['0.1th'][Month]) | (Tend >= E_percentiles_ind_6hr['99.9th'][Month]):
               extreme_storms_BF_RF.append(Data)
           else:
               pass
        elif (Tendency_Window == 18):
            if (Tend <= E_percentiles_ind_18hr['0.1th'][Month]) | (Tend >= E_percentiles_ind_18hr['99.9th'][Month]):
                extreme_storms_BF_RF.append(Data)
            else:
                pass
    
    extreme_storms_BF_RF = pd.DataFrame(extreme_storms_BF_RF) #in the 1800s
    extreme_storms_BF_RF = extreme_storms_BF_RF.set_index('Date')
    extreme_storms_BF_RF.index= pd.to_datetime(extreme_storms_BF_RF.index,format='mixed',dayfirst = True)
    Extreme_Winter_Storms = ChatJR.filter_dataframe_by_months(extreme_storms_BF_RF,Months_Inc)


    Extreme_Winter_Storms['Time Between Potential Storms'] = Extreme_Winter_Storms.index.diff().days
    Extreme_Winter_Storms['Storms'] = (Extreme_Winter_Storms['Time Between Potential Storms'] > 1).cumsum()+1

    es_cols = Extreme_Winter_Storms.columns

    Extreme_Winter_Storms =Extreme_Winter_Storms.rename(columns = {P_MSLP_SD.columns[0] :'MSLP (hPa)'})
    Extreme_Winter_Storms =Extreme_Winter_Storms.rename(columns = {es_cols[1] :'Hours To Next MSLP Reading'})
    Extreme_Winter_Storms =Extreme_Winter_Storms.rename(columns = {es_cols[2] :'MSLP Change (hPa)'})
    Extreme_Winter_Storms =Extreme_Winter_Storms.rename(columns = {es_cols[3] :'Days between Storms'})
    Extreme_Winter_Storms =Extreme_Winter_Storms.rename(columns = {es_cols[4] :'Storm ID'})
    Extreme_Winter_Storms =Extreme_Winter_Storms.rename(columns = {es_cols[5] :'Next MSLP Reading (hPa)'})
    Extreme_Winter_Storms = Extreme_Winter_Storms.reset_index().set_index('Storm ID')
    
    Extreme_Winter_Storms = Extreme_Winter_Storms[['Date','MSLP (hPa)', 'Hours To Next MSLP Reading', 'Next MSLP Reading (hPa)', 'MSLP Change (hPa)']] 
    Extreme_Winter_Storms['Hours To Next MSLP Reading'] =Extreme_Winter_Storms['Hours To Next MSLP Reading'].round(0)
    if (Tendency == True):
        return(Winter_Storms,Extreme_Winter_Storms,[percentiles_ind_6hr,percentiles_ind_18hr, E_percentiles_ind_6hr,E_percentiles_ind_18hr])
    else:
        return(Winter_Storms,Extreme_Winter_Storms)

In [ ]:
def Storm_Finder_With_Rain(Storms_Before, Extreme_Storms_Before, Raindays, Rainfall, Rainfal_Target):

    Storms =Storms_Before
    Extreme_Storms = Extreme_Storms_Before

    Storms = Storms.set_index('Date')
    Extreme_Storms = Extreme_Storms.set_index('Date')
    #filter raindays into it
    Storms = pd.merge(Storms, Raindays, how='inner', left_index=True, right_index=True)
    Storms = pd.merge(Storms, Rainfall, how='inner', left_index=True, right_index=True)
    Storms.index.name = 'Date'

    #filter raindays into it
    Extreme_Storms = pd.merge(Extreme_Storms, Raindays, how='inner', left_index=True, right_index=True)
    Extreme_Storms = pd.merge(Extreme_Storms, Rainfall, how='inner', left_index=True, right_index=True)
    Extreme_Storms.index.name = 'Date'

    #Identify if rainfall and raindays is in there
    Storms = Storms[Storms[Raindays.columns[0]] == 1]
    Extreme_Storms = Extreme_Storms[Extreme_Storms[Raindays.columns[0]] == 1]
    
    #Find the time between storms again
    Storms['Days between Storms'] = Storms.index.diff().days
    Extreme_Storms['Days between Storms'] = Extreme_Storms.index.diff().days
    
    # Identify new groups where the difference is not 1 day
    Storms['Storm ID'] = (Storms['Days between Storms'] > 1).cumsum()+1
    Extreme_Storms_RF =Storms[Storms['RAINFALL'] >=Rainfal_Target]

    #Extreme Storms on a Rainfall basis
    Extreme_Storms = Extreme_Storms.combine_first(Extreme_Storms_RF)
   
    Extreme_Storms['Days between Storms'] = Extreme_Storms.index.diff().days
    Extreme_Storms['Storm ID'] = (Extreme_Storms['Days between Storms'] > 1).cumsum()+1


    Storms =  Storms.reset_index().set_index('Storm ID')
    Extreme_Storms =  Extreme_Storms.reset_index().set_index('Storm ID')
    Extreme_Storms = Extreme_Storms[['Date','MSLP (hPa)','RAINFALL', 'Hours To Next MSLP Reading','Next MSLP Reading (hPa)','MSLP Change (hPa)']]
    Extreme_Storms = Extreme_Storms.rename(columns={"RAINFALL": "Rainfall"})
    Storms = Storms.rename(columns={"RAINFALL": "Rainfall"})
    
    return(Storms,Extreme_Storms)